In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from langchain.globals import set_verbose, set_debug

set_debug(False)
set_verbose(False)

In [1]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.vectorstores import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.retrievers.document_compressors import LLMChainFilter
from langchain.retrievers import ContextualCompressionRetriever
from langchain_core.messages import AIMessage, HumanMessage

In [2]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [3]:
persist_directory = 'all_docs_zotero/chroma/'
embedding = OpenAIEmbeddings()

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

C:\Users\pouri\anaconda3\envs\langchain_env\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [4]:
metadata_field_info = [
    AttributeInfo(
        name="Publication Year",
        description="The year that the paper was published.",
        type="integer",
    ),
    AttributeInfo(
        name="Date Added",
        description="The year that the paper was added to the collection.",
        type="integer",
    ),
    AttributeInfo(
        name="Author",
        description="Authors of the paper, it could be couple of people.",
        type="string",
    ),
    AttributeInfo(
        name="Title", 
        description="Title of the paper that the paper is about.", 
        type="string",
    ),
]

document_content_description = "Brain Heart Interconnectome (BHI) research papers"
llm = ChatOpenAI(temperature=0)

retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    #search_kwargs={"k": 10}
    #enable_limit=True,
)

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter,
    base_retriever=retriever
)

In [5]:
### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, compression_retriever, contextualize_q_prompt
)

In [6]:
### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know.

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [7]:
### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [8]:
conversational_rag_chain.invoke(
    {"input": "What can you tell me about consort?"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)['answer']

Parent run 30650b85-4450-4ca1-9f54-aff8120c88ca not found for run f44674b4-73e7-496c-8ed6-7467cd93acf0. Treating as a root run.


'The CONSORT Statement is a key tool for achieving adequate reporting in research. It includes a checklist and flow diagram that journals can incorporate into their review processes to ensure adherence. Endorsement of CONSORT by journals is crucial for its effectiveness, and it is defined as editorial statements endorsing CONSORT, requirements in journal instructions to authors, or requirements for authors to submit CONSORT checklists and flow diagrams. Studies have shown that journals endorsing CONSORT have higher completeness of reports compared to non-endorsing journals.'

In [9]:
conversational_rag_chain.invoke(
    {"input": "Tell me more"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)['answer']

Parent run 254eb357-0daa-40fe-81d7-d06786ea438c not found for run f2587c02-9513-419e-8ca7-72d1766c6f1b. Treating as a root run.


'The CONSORT Statement, which stands for Consolidated Standards of Reporting Trials, was developed to improve the reporting of randomized controlled trials (RCTs). It consists of a 25-item checklist and a flow diagram that provide guidance on how to report various aspects of an RCT, such as the study design, participant recruitment, interventions, outcomes, and statistical analysis. Adherence to the CONSORT guidelines helps ensure transparency and completeness in reporting, which is essential for evaluating the validity and reliability of trial results. Journals are encouraged to endorse the CONSORT guidelines and require authors to follow them when submitting RCT manuscripts for publication.'

In [10]:
conversational_rag_chain.get_graph().print_ascii()

                         +-----------------------------+                      
                         | Parallel<chat_history>Input |                      
                         +-----------------------------+                      
                                ***            ***                            
                              **                  **                          
                            **                      **                        
            +------------------------+          +-------------+               
            | Lambda(_enter_history) |          | Passthrough |               
            +------------------------+          +-------------+               
                                ***            ***                            
                                   **        **                               
                                     **    **                                 
                        +---------------------------

In [13]:
conversational_rag_chain.get_prompts()

[PromptTemplate(input_variables=['page_content'], template='{page_content}'),
 ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.\n\n{context}")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])]